In [ ]:
import os

from joblib import delayed

from rule.distance_FV import DistanceCompareFVGenerator
from rule.distance_NI import DistanceNIGenerator
from rule.quantity_FV import QuantityCompareFVGenerator
from rule.quantity_NI import QuantityNIGenerator
from rule.volume_FV import VolumeCompareFVGenerator
from rule.volume_NI import VolumeNIGenerator
from utils.io import enum_files
from utils.parallel import ParallelTqdm

In [ ]:
def generate_questions(
        generator_class,
        scene_stat_json_file: str,
        output_json_file: str,
        n_questions: int,
):
    generator_class(
        scene_stat_json_file=scene_stat_json_file,
        output_json_file=output_json_file,
    ).generate(n_questions=n_questions)

In [ ]:
export_path = '../output'
scene_json_dir = '../NUMINA-dataset/ScanNet-scene_stats'

tasks = [
    (generator, scene_json, os.path.join(export_path, filename), 6)
    for scene_json in enum_files(scene_json_dir)
    for generator, filename in [
        (VolumeNIGenerator, 'NUM-volume-NI.json'),
        (VolumeCompareFVGenerator, 'NUM-volume_compare-FV.json'),
        (QuantityNIGenerator, 'NUM-count-NI.json'),
        (QuantityCompareFVGenerator, 'NUM-count_compare-FV.json'),
        (DistanceNIGenerator, 'NUM-distance-NI.json'),
        (DistanceCompareFVGenerator, 'NUM-distance_compare-FV.json'),
    ]
]

ParallelTqdm(n_jobs=8)(
    delayed(generate_questions)(generator_class, scene_json, output_json, n_questions)
    for generator_class, scene_json, output_json, n_questions in tasks[::5]
)
